In [ ]:
# 1. Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# 2. Загрузка датасета
df = pd.read_csv('S05-hw-dataset.csv')

# 3. Анализ датасета
print(df.head())

print(df.info())

print(df.describe())

target_distribution = df['default'].value_counts(normalize=True)
print(target_distribution)

# Датасет содержит 3000 объектов (клиентов) и 17 признаков.
# аномалии: savings_balance, присутсвуют отрицательные значения
# Целевая переменная распределена с умеренным дисбалансом: 59% клиентов без дефолта (класс 0) и 41% с дефолтом (класс 1).

In [ ]:
# 1. Выделяем матрицу признаков X и вектор таргета y
X = df.drop(columns=['default', 'client_id'])
y = df['default']

# 2. Проверяем, что debt_to_income в диапазоне [0, 1]
debt_invalid = X[(X['debt_to_income'] < 0) | (X['debt_to_income'] > 1)]
if not debt_invalid.empty:
    print(f"Найдено {len(debt_invalid)} записей с debt_to_income вне диапазона [0, 1]")

# Проверка признаков
if X.select_dtypes(include=['object']).empty:
    print("Все признаки числовые")
else:
    print("Есть нечисловые признаки")

In [ ]:
# 1. Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

# 2. Бейзлайн-модель
baseline = DummyClassifier(strategy="most_frequent")
baseline.fit(X_train, y_train)

# 3. Оценка бейзлайна
y_pred = baseline.predict(X_test)
accuracy1 = accuracy_score(y_test, y_pred)

y_pred_proba = baseline.predict_proba(X_test)[:, 1]
roc_auc1 = roc_auc_score(y_test, y_pred_proba)

print(accuracy1)
print(roc_auc1)

# Бейзлайн предсказывает самый частый класс (не дефолт). Это точка отсчёта для оценки сложности задачи.
# Точка отсчёта показывает, нужно ли вообще строить сложную модель: 
# если она не превосходит простое угадывание самого частого класса, 
# то задача либо слишком сложна, либо данные не содержат полезных закономерностей.

In [ ]:
# Создание пайплайна
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=42))
])

# Подбор параметра C
param_grid = {'logreg__C': [0.01, 0.1, 1.0, 10.0]}
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# Лучшая модель
best_model = grid_search.best_estimator_

# Предсказания на тестовой выборке
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Метрики
accuracy2 = accuracy_score(y_test, y_pred)
roc_auc2 = roc_auc_score(y_test, y_pred_proba)
best_param = grid_search.best_params_['logreg__C']
print(accuracy2,roc_auc2,best_param)
# ROC-кривая
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC-кривая (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Случайный классификатор')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('./figures/roc_curve.png', dpi=100, bbox_inches='tight')

In [ ]:


results = pd.DataFrame({
    'Модель': ['DummyClassifier', 'LogisticRegression'],
    'Accuracy': [accuracy1, accuracy2],
    'ROC-AUC': [roc_auc1, roc_auc2]
})

print("Сравнение моделей:")
print(results.to_string(index=False))



Бейзлайн-модель показывает accuracy 59% и ROC-AUC 0.5, что соответствует случайному угадыванию.
 Логистическая регрессия с оптимальной регуляризацией (C=1.0) значительно превосходит бейзлайн: 
 accuracy выросла до 80%, а ROC-AUC достиг 0.876, что указывает на отличную разделяющую способность модели. 
 Умеренная регуляризация (C=1.0) обеспечила наилучший баланс между недообучением и переобучением. Д
 ля данной задачи логистическая регрессия является эффективным решением, так как существенно улучшает предсказания, 
 остаётся интерпретируемой и показывает высокое качество на тестовой выборке.